In [134]:
# importar librerias
import pandas as pd

In [135]:
# Extraer los mejores 100 juegos de la base de datos
Lista = []
for i in range(1, 11):
    archivo = "Data/RegistroJuegos_" + str(i) + ".csv"
    RegistroJuegos = pd.read_csv(archivo)
    Lista.append(RegistroJuegos)

# Concatenar RegistroJuegos
RegistroJuegos = pd.concat(Lista, axis=0)
RegistroJuegos.shape

# Concatenar InfoJuegos
Lista2 = []
for i in range(1, 6):
    archivo = "Data/InfoJuegos_" + str(i) + ".csv"
    InfoJuegos = pd.read_csv(archivo)
    Lista2.append(InfoJuegos)

InfoJuegos = pd.concat(Lista2, axis=0)

# Concatenar PreferenciaJuegos
Lista3 = []
for i in range(1, 6):
    archivo = "Data/PreferenciaUser_" + str(i) + ".csv"
    PreferenciaUser = pd.read_csv(archivo)
    Lista3.append(PreferenciaUser)

PreferenciaUser = pd.concat(Lista3, axis=0)

# Creamos una lista vacia para guardar los juegos. Seguido de un loop para extraer la informacion de
# los 10 archivos que anteriormente creamos. Se concatenan los archivos y se guarda en la lista vacia.
# Se repite este proceso para los archivos de InfoJuegos y PreferenciaUser.

In [136]:
# Ensure 'tipo' column exists before filtering
TipoGame = RegistroJuegos[RegistroJuegos['Tipo'] == 'game']
RegistroJuegos = TipoGame.sort_values(['Tipo'])
print(f'Cantidad de filas/columnas de RegistroJuegos: {RegistroJuegos.shape}')

TipoGame2 = InfoJuegos[InfoJuegos['Tipo'] == 'game']
InfoJuegos = TipoGame2.sort_values(['Tipo'])
print(f'Cantidad de filas/columnas de InfoJuegos: {InfoJuegos.shape}')

print(f'Cantidad de filas/columnas de PreferenciaUser: {PreferenciaUser.shape}')

# Se filtran los datos por tipo de juego y se utiliza sort_values para ordenar los datos.
# Se imprime la cantidad de filas y columnas de cada archivo. Asi tenemos una idea
# de la cantidad de datos que estamos manejando.

Cantidad de filas/columnas de RegistroJuegos: (543, 15)
Cantidad de filas/columnas de InfoJuegos: (252, 15)
Cantidad de filas/columnas de PreferenciaUser: (1000, 15)


In [137]:
# Visualizar encabezados de los dataframes
print(RegistroJuegos.head(0))
print(InfoJuegos.head(0))
print(PreferenciaUser.head(0))

# Verificamos los encabezados de los dataframes para saber que columnas tenemos en cada uno.
# Esto nos ayudara a saber que podemos utilizar para hacer el merge.

Empty DataFrame
Columns: [AppID, Nombre, Tipo, Descripcion, Desarrollador, Editor, FechaSalida, Genero, Categoria, Lenguajes, Gratis, EdadRequerida, Moneda, PrecioInicial, Plataforma]
Index: []
Empty DataFrame
Columns: [AppID, Nombre, Tipo, Desarrollador, Editor, FechaSalida, Genero, Categoria, Gratis, Plataforma, Popularidad, TamanoGB, SoporteMods, FrecuenciaActualizaciones, ComprasEnLaApp]
Index: []
Empty DataFrame
Columns: [ID, Usuario, JuegoFavorito, CalificacionUsers, Descargas, Resenas, TiempoJuegoHoras, AccesoAnticipado, CompatibleConsola, Popularidad, Edad, PaísUser, UserOnline, TotalUsers, SoporteSteamDeck]
Index: []


In [138]:
# mejor juego calificado
JuegosPopulares = PreferenciaUser.groupby('JuegoFavorito')['CalificacionUsers'].mean().reset_index()
JuegosPopulares = round(JuegosPopulares.sort_values('CalificacionUsers', ascending=False),1)
print(JuegosPopulares.shape)

# Organizar los encabezados
# ID, Nombre, Desarrollador, Precio, Moneda, Calificacion, Resena, Genero, categoria, Plataforma, FechaLanzamiento, descargas, UserOnline, SoporteMods, comprasEnLaApp

# Se crea un nuevo dataframe llamado JuegosPopulares, donde se agrupan los juegos por su calificacion promedio.
# luego, se ordenan los juegos con una mayor o menor calificacion, esto se redondea a un decimal para que
# tenga mas coherencia.

(400, 2)


In [139]:
# Mostrar JuegosPopulares en la lista RegistroJuegos
RegistroJuegos = RegistroJuegos.merge(JuegosPopulares, left_on='Nombre', right_on='JuegoFavorito', how='inner', suffixes=('_Registro', '_Popular'))

# Ensure columns exist before selecting
Encabezados = ['AppID', 'Nombre', 'Desarrollador', 'PrecioInicial', 'Moneda', 'CalificacionUsers', 'Resenas', 'Genero', 'TamanoGB', 'Plataforma', 'FechaSalida', 'Descargas', 'UserOnline', 'SoporteMods', 'ComprasEnLaApp']
Existentes = [col for col in Encabezados if col in RegistroJuegos.columns]
AnalisisJuegos = RegistroJuegos[Existentes]

AnalisisJuegos.head(10)

# Se realiza un merge de los juegos populares con los juegos de RegistroJuegos. Luego, se seleccionan las columnas
# que se van a utilizar para el analisis de los juegos. y en Existentes se guardan las columnas que se van a utilizar
# para el analisis. Finalmente, se muestra la info de los 10 primeros juegos.

,AppID,Nombre,Desarrollador,PrecioInicial,Moneda,CalificacionUsers,Genero,Plataforma,FechaSalida
0,1473760,Beyond the Storm,LeandroGabriel.net,11.99,USD,3.9,RPG,windows,"3 Oct, 2022"
1,2430820,Squab,"Guimaraf Studio, Nibb Games",1.99,USD,2.4,Action,linux,"15 May, 2024"
2,2430800,Laser Survivor,Andrey Kopylov,To be announced,To be announced,3.2,Casual,mac,To be announced
3,2430770,Violet Bricks,Vektro Digital,3.99,USD,3.6,RPG,linux,"30 May, 2023"
4,2430790,Phonk Cats,Soulwarmers Games,1.99,USD,3.9,RPG,linux,"26 May, 2023"
5,2430760,Family Arcade,Family Nights,0.00,NaN,3.3,Adventure,windows,"1 Dec, 2023"
6,2430730,My Child Lebensborn Remastered,Sarepta Studio AS,7.99,USD,3.0,RPG,windows,"18 Jul, 2023"
7,2430710,Operator: Observation,Stronggaming Studios,To be announced,To be announced,3.8,Adventure,linux,To be announced
8,2430690,Clover Day's Plus,ALcot,34.99,USD,3.5,Action,mac,"18 Aug, 2023"
9,2430680,Hope is Gone,Helemon,3.99,USD,2.6,Casual,linux,"15 Feb, 2024"


In [140]:
# Filtrar columnas en InfoJuegos
# ['TamanoGB', 'Plataforma', 'SoporteMods', 'ComprasEnLaApp']
InfoJuegos = InfoJuegos[['Nombre', 'TamanoGB', 'Plataforma', 'SoporteMods', 'ComprasEnLaApp']]
InfoJuegos = pd.DataFrame(InfoJuegos)

# Filtrar columnas en PreferenciaUser
# ['Resenas', 'Descargas', 'UserOnline']
PreferenciaUser = PreferenciaUser[['JuegoFavorito', 'Resenas', 'Descargas', 'UserOnline']]
PreferenciaUser = pd.DataFrame(PreferenciaUser)

# Solo queda unir las columnas restantes de InfoJuegos y PreferenciaUser con el dataframe de AnalisisJuegos.
# Para ello, se filtran las columnas que se van a utilizar en InfoJuegos y PreferenciaUser. Luego, se crea un nuevo
# dataframe con la informacion filtrada.

In [141]:
# Crear clave para unir los dataframes
InfoJuegos['key'] = InfoJuegos['Nombre']
PreferenciaUser['key'] = PreferenciaUser['JuegoFavorito']

result = pd.merge(InfoJuegos, PreferenciaUser, on="key")
result.head(5)

# eliminar columnas key y JuegoFavorito
result = result.drop(columns=['key', 'JuegoFavorito'])
result.head(5)

# Luego de filtrar, se crea una clave para unir los dataframes. Se unen los dataframes con la clave y se eliminan
# las columnas que no se van a utilizar.

,Nombre,TamanoGB,Plataforma,SoporteMods,ComprasEnLaApp,Resenas,Descargas,UserOnline
0,Beyond the Storm,64,windows,No,No,Mayormente Positivo,90347,9314
1,Infernal Apocalypse: Rise of the Underworld,51,mac,No,Si,Muy positivo,41788,1250
2,Infernal Apocalypse: Rise of the Underworld,51,mac,No,Si,Mayormente Negativo,12071,2103
3,Infernal Apocalypse: Rise of the Underworld,51,mac,No,Si,Mayormente Negativo,8125,5995
4,Infernal Apocalypse: Rise of the Underworld,51,mac,No,Si,Mixto,70732,5367


In [142]:
# Unir dataframes
AnalisisJuegos = pd.concat([AnalisisJuegos, result], axis=1)

# Eliminar columnas duplicadas
AnalisisJuegos = AnalisisJuegos.loc[:, ~AnalisisJuegos.columns.duplicated()]

# Unir valores de juegos duplicados
AnalisisJuegos = AnalisisJuegos.groupby('Nombre').first().reset_index()

# Cambiar formato de numeros en columnas AppID, Descargas, UserOnline y TamanoGB
AnalisisJuegos['AppID'] = AnalisisJuegos['AppID'].apply(lambda x: '{:.0f}'.format(x))
AnalisisJuegos['Descargas'] = AnalisisJuegos['Descargas'].apply(lambda x: '{:.0f}'.format(x))
AnalisisJuegos['UserOnline'] = AnalisisJuegos['UserOnline'].apply(lambda x: '{:.0f}'.format(x))
AnalisisJuegos['TamanoGB'] = AnalisisJuegos['TamanoGB'].apply(lambda x: '{:.0f}'.format(x))

AnalisisJuegos.head(5)

# Finalmente, se unen los dataframes con la funcion concat. Se eliminan las columnas duplicadas utilizando loc.
# esto permite seleccionar las columnas que no estan duplicadas. Con :, ~ se eliminan las columnas duplicadas.
#
# Luego, se agrupan los valores de los juegos duplicados y se cambia el formato de los numeros en las columnas AppID,
# Descargas, UserOnline y TamanoGB. Esto se realiza con la funcion apply y lambda para darle formato a los numeros.

,Nombre,AppID,Desarrollador,PrecioInicial,Moneda,CalificacionUsers,Genero,Plataforma,FechaSalida,TamanoGB,SoporteMods,ComprasEnLaApp,Resenas,Descargas,UserOnline
0,(the) Gnorp Apologue,1473350,Myco,6.99,USD,2.6,RPG,windows,"14 Dec, 2023",92,Si,No,Mixto,21805,3438
1,100 hidden birds,1473050,"Anatoliy Loginovskikh, Аlexandr Telminov",1.99,USD,4.7,Adventure,mac,"2 Dec, 2020",18,No,Si,Mayormente Positivo,61826,3125
2,A Game of Gods: Infinium,2430020,Studio Zenn,To be announced,To be announced,4.6,Casual,windows,To be announced,34,Si,Si,Mayormente Negativo,99815,9357
3,A Pigeon Detective Game: Twin Beaks,1875230,Alex Taylor,To be announced,To be announced,2.4,Strategy,mac,To be announced,65,No,Si,Mayormente Positivo,33932,1548
4,A.V.A Global,1473480,Valiant,0.00,None,1.9,Indie,mac,"23 Aug, 2022",23,Si,Si,Mayormente Negativo,37633,3603


In [144]:
# Exportar dataframe a archivo csv
AnalisisJuegos.to_csv('AnalisisJuegos.csv', index=False)